In [5]:
from keras.layers import Layer, Dense, Input, Conv2D, UpSampling2D, RepeatVector
from keras.layers import Flatten, Reshape, Conv2DTranspose, MaxPooling2D, Dropout, UpSampling2D
from keras.layers.merge import concatenate
from keras.engine.topology import Layer
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.datasets import cifar10, cifar100
from keras.utils import plot_model
from keras import backend as K
from keras import utils
from keras.layers import Lambda

input_shape = (32, 32, 1)
kernel_size = 3
layer_filters = [64, 128, 256]
inputs = Input(shape=input_shape, name='encoder_input')
x = inputs
latent_dim = 256
for filters in layer_filters:
    x = Conv2D(filters=filters,
            kernel_size=kernel_size,
            strides=2,
            activation='relu',
            padding='same')(x)
    x = Dropout(0.25)(x)
shape = K.int_shape(x)
# generate a latent vector
x = Flatten()(x)
latent = Dense(latent_dim, name='latent_vector')(x)
# instantiate encoder model
encoder = Model(inputs, latent, name='encoder')
# build the decoder model
latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(shape[1]*shape[2]*shape[3])(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)
for filters in layer_filters[::-1]:
    x = Conv2DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        strides=2,
                        activation='relu',
                        padding='same')(x)
outputs = Conv2DTranspose(filters=2,
                        kernel_size=kernel_size,
                        activation='relu',
                        padding='same',
                        name='decoder_output')(x)
# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
model = Model(inputs, decoder(encoder(inputs)))

In [6]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [7]:
# Network parameters
input_shape = (32, 32, 1)
batch_size = 20
kernel_size = 3
dropout = 0.1
filters = [64, 128, 256]


# Traditional convolution network
inputs = Input(shape=input_shape)
x = inputs
# Three layers of Conv2D Dropout Max Pooling 2D
for filtr in filters:
    x = Conv2D(filters = filtr,
              kernel_size = kernel_size,
              padding = "same",
              activation = "relu")(x)
    x = MaxPooling2D()(x)
    
y_input = Input(shape=input_shape)
y = y_input
for m_filter in filters:
    y = Conv2D(filters = m_filter,
              kernel_size = kernel_size,
              padding = "same",
              activation = "relu",
              dilation_rate=2)(y)
    y = MaxPooling2D()(y)

y = concatenate([x, y])
    
for filtr in filters[::-1]:
    x = Conv2D(filters = filtr,
              kernel_size = kernel_size,
              padding = "same",
              activation = "relu")(x)
    x = UpSampling2D((2,2))(x)
output = Conv2D(2, (3, 3), activation='sigmoid', padding='same')(x)
model = Model(inputs, output)

In [8]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)